In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, lit, count
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

# Initialize Spark Session (Local Mode)
spark = SparkSession.builder \
    .appName("Airbnb_Data_Prep") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/24 11:02:28 WARN Utils: Your hostname, nnnnnn.local, resolves to a loopback address: 127.0.0.1; using 10.62.115.9 instead (on interface en0)
25/11/24 11:02:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/24 11:02:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Version: 4.0.1


In [12]:
# ==========================================
# Cell 2: Load & Clean Listings (The Metadata)
# ==========================================
# We use multiLine=True because your description field has newlines (<br />)
listings_df = spark.read.csv(
    "../data/listings.csv",
    header=True,
    inferSchema=True,
    multiLine=True,
    quote='"',
    escape='"'
)

# Select only what we need for the App UI
# converting "$66.00" -> 66.0 using regex
clean_listings = listings_df.select(
    col("id").alias("listing_id_raw"),
    col("name"),
    col("host_name"),
    col("neighbourhood_cleansed").alias("neighborhood"),
    regexp_replace(col("price"), "[\$,]", "").cast(FloatType()).alias("price"),
    col("review_scores_rating").cast(FloatType()).alias("avg_rating"),
    col("number_of_reviews").cast(IntegerType()).alias("number_of_reviews"),
).dropna(subset=["listing_id_raw"])



print("Listings Count:", clean_listings.count())
clean_listings.show(5)

<>:21: SyntaxWarning: invalid escape sequence '\$'
<>:21: SyntaxWarning: invalid escape sequence '\$'
/var/folders/vj/8wn3h6116s1g_mdd5ktxzjt00000gn/T/ipykernel_20520/4074167678.py:21: SyntaxWarning: invalid escape sequence '\$'
  regexp_replace(col("price"), "[\$,]", "").cast(FloatType()).alias("price"),


Listings Count: 36111
+--------------+--------------------+---------+------------+-----+----------+-----------------+
|listing_id_raw|                name|host_name|neighborhood|price|avg_rating|number_of_reviews|
+--------------+--------------------+---------+------------+-----+----------+-----------------+
|      40824219|Room close to  Ma...|    Sevil|   Sunnyside| 66.0|      4.81|               16|
|      40833186|Soho LES East vil...| Patricio|      Nolita| NULL|      NULL|                0|
|      40837137|Sunset Park - Qui...|     Xuan| Sunset Park| NULL|       5.0|                1|
|      40838018|Cozy One Bedroom ...|   Jamila|Clinton Hill| NULL|       5.0|                1|
|      40839416|🪴XL dojo 🌾 shar...|   Haffro|East Village| 76.0|      4.95|               20|
+--------------+--------------------+---------+------------+-----+----------+-----------------+
only showing top 5 rows
